In [ ]:
# Set langsmith tracing
from langchain_teddynote import logging
from configs import llm
# input your project name
logging.langsmith("Practice_Streaming_Stdout")

## 1.Streaming response
* `StateGraph.stream`을 통해 구현 가능함
#### 주요 기능

* 입력된 설정에 따라 그래프 실행을 스트리밍 방식으로 처리
* 다양한 스트리밍 모드 지원 (values, updates, debug)
* 콜백 관리 및 오류 처리
* 재귀 제한 및 중단 조건 처리

#### 스트리밍 모드

* values: 각 단계의 현재 상태 값 출력
* updates: 각 단계의 상태 업데이트만 출력
* debug: 각 단계의 디버그 이벤트 출력
####  Parms
  * input : 그래프에 대한 입력
    ```python
    (Union[dict[str, Any], Any])
    ```
  * config : 실행 구성
    ```python
    (Optional[RunnableConfig])
    ```
  * stream_mode : 출력 스트리밍 모드
    ```python
    (Optional[Union[StreamMode, list[StreamMode]]])
    ```
  * output_keys : 스트리밍할 키 
    ```python
    (Optional[Union[str, Sequence[str]]])
    ```
  * interrupt_before : 실행 전에 중단할 노드
    ```python
    (Optional[Union[All, Sequence[str]]])
    ```
  * interrupt_after : 실행 후에 중단할 노드
    ```python
    (Optional[Union[All, Sequence[str]]])
    ```
  * debug : 디버그 정보 출력 여부 
    ```python
    (Optional[bool])
    ```
  * subgraphs : 하위 그래프 스트리밍 여부
    ```python
    (bool)
    ```
#### Return
* 그래프의 각 단계 출력. 출력 형태는 stream_mode에 따라 다름
    ```python
    Iterator[Union[dict[str, Any], Any]]
    ``` 

In [ ]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict

from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.graphs import visualize_graph
from langchain_teddynote.tools import GoogleNews

In [ ]:
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]
    dummy_data: Annotated[str, "dummy"]
    
# 도구 정의
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

# 호출된 도구를 모델과 바인딩
tools = [search_keyword]
llm_with_tools = llm.bind_tools(tools) # 도구와 LLM 결합
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {
        "messages": [llm_with_tools.invoke(state["messages"])], # 도구 바인딩된 모델 호출
        "dummy_data": "[chatbot] 호출, dummy data",  # 테스트를 위하여 더미 데이터를 추가합니다.
    }

In [ ]:
# 상태그래프 생성 및 노드,엣지 연결
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)


# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=tools)

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

In [ ]:
########## 5. 그래프 컴파일 ##########

# 그래프 빌더 컴파일
graph = graph_builder.compile()

########## 6. 그래프 시각화 ##########
# 그래프 시각화
visualize_graph(graph)


In [ ]:
# Set Input Config
from langchain_core.runnables import RunnableConfig

# 질문
question = "2024년 노벨 문학상 관련 뉴스를 알려주세요."

# 초기 입력 상태를 정의
input = State(
    dummy_data="테스트 문자열",
    messages=[("user", question)]
)

# config 설정
config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
    tags=["my-tag"],  # Tag
)


#### output_keys
* output_keys 옵션은 스트리밍할 키를 지정하는 데 사용됩니다.

* list 형식으로 지정할 수 있으며, channels 에 정의된 키 중 하나 여야 합니다.

* 팁(tip)

    * 매 단계마다 출력되는 State key 가 많은 경우, 일부만 스트리밍하고 싶은 경우에 유용합니다.

In [ ]:
for event in graph.stream(input=input, config=config):
    for key, value in event.items():
        print(f"\n[ {key} ]\n")
        # value 에 messages 가 존재하는 경우
        if "messages" in value:
            messages = value["messages"]
            # 가장 최근 메시지 1개만 출력합니다.
            value["messages"][-1].pretty_print()


In [ ]:
# 마지막 요소의 모델의 답변만 출력
for event in graph.stream(
    input=input,
    config=config,
    output_keys=["messages"],  # messages 만 출력
    ):
    for key, value in event.items():
        # messages 가 존재하는 경우
        if value and "messages" in value:
            # key 는 노드 이름
            print(f"\n[ {key} ]\n")
            # messages 의 마지막 요소의 content 를 출력합니다.
            print(value["messages"][-1].content)


#### `stream_mode` 옵션
* stream_mode 옵션은 스트리밍 출력 모드를 지정하는 데 사용됩니다.

#### values: 각 단계의 현재 상태 값 출력
#### updates: 각 단계의 상태 업데이트만 출력 (기본값)
`stream_mode = "values"`
* values 모드는 각 단계의 현재 상태 값을 출력합니다.

### 참고
```python
event.items()
```

* key: State 의 key 값
* value: State 의 key 에 대한하는 value

In [ ]:
# values 모드로 스트리밍 출력
for event in graph.stream(
    input=input,
    stream_mode="values",  # 기본값, updates
):
    for key, value in event.items():
        # key 는 state 의 key 값
        print(f"\n[ {key} ]\n")
        if key == "messages":
            print(f"메시지 개수: {len(value)}")
            print(value)
    print("===" * 10, " 단계 ", "===" * 10)


#### interrupt_before 와 interrupt_after 옵션
* interrupt_before 와 interrupt_after 옵션은 스트리밍 중단 시점을 지정하는 데 사용됩니다.

* interrupt_before
    * 지정된 노드 이전에 스트리밍 중단
* interrupt_after
    * 지정된 노드 이후에 스트리밍 중단

In [ ]:
# interrupt를 이용해서 특정 노드 앞에서 중단하기
for event in graph.stream(
    input=input,
    config=config,
    stream_mode="updates",  # 기본값
    interrupt_before=["tools"],  # tools 노드 이전에 스트리밍 중단
):
    for key, value in event.items():
        # key 는 노드 이름
        print(f"\n[{key}]\n")

        # value 는 노드의 출력값
        if isinstance(value, dict):
            print(value.keys())
            if "messages" in value:
                print(value["messages"])

        # value 에는 state 가 dict 형태로 저장(values 의 key 값)
        if "messages" in value:
            print(f"메시지 개수: {len(value['messages'])}")
    print("===" * 10, " 단계 ", "===" * 10)


In [ ]:
# interrupt 이용해서 특정 노드 이후 중단
for event in graph.stream(
    input=input,
    config=config,
    stream_mode="updates",
    interrupt_after=["tools"],  # tools 실행 후 interrupt
):
    for value in event.values():
        # key 는 노드 이름
        print(f"\n[{key}]\n")

        if isinstance(value, dict):
            # value 는 노드의 출력값
            print(value.keys())
            if "messages" in value:
                print(value["messages"])

        # value 에는 state 가 dict 형태로 저장(values 의 key 값)
        if "messages" in value:
            print(f"메시지 개수: {len(value['messages'])}")
